# [0. Settings](#0) 
<br></br>
## [0.1 Librerie](#0.1)

<font size=3>

Le librerie importate, non presenti nell'altro notebook sono:
* `StanfordNERTagger`: libreria sviluppata da Stanford che permette di effettuare un NER sulle recensioni.
* `sner`: dato che l'operazione in locale è lenta, con questa libreria è possibile accedere al server di Stanford:
    * `Ner`: la funzione si connette al serve per effettuare il NER.
    * `POSClient`: stesso meccanismo utilizzato per il POS tagging visto che anche questo metodo è lento in locale.
* `StanfordPOSTagger`: per il NER è possibile connettersi al server direttamente con la libreria di Stanford, per il POS tagging è più comodo passare per `nltk`, al cui interno è presente la medesima funzione, ed è più facile accedere al server.
* `re`: è una libreria per la modifica di stringhe, permette di fare sostituzioni e si lavorare con alcuni pattern.

In [1]:
from collections import Counter
from iteration_utilities import deepflatten
from tqdm import tqdm

from nltk.tag import StanfordNERTagger
from sner import Ner, POSClient

from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string

from nltk.tag import StanfordPOSTagger

import gensim
import nltk
import os
import pandas as pd
import pickle
import re
import stanfordnlp

os.chdir('D:/Scraping')

<br></br>
## [0.2 File Reading](#0.2)

In [2]:
# Recensioni e titoli
imdb_df = pd.read_csv("imdb_df.csv", delimiter=';')
movies = pd.read_csv("movies titles.csv", delimiter=';')
info = pd.read_csv("imdb_info.csv", delimiter="\t")

<br></br>
<br></br>
# [1. Pre-processing](#1)
<br></br>
## [1.1 Merge Titoli](#1.1)

<font size=3>

Si crea un dizionario coppia-valore con rispettivamente i titoli geolocalizzati e i titoli originali. Si utilizza un dizionario in quanto è il metodo più veloce per assegnare le corrispondenze corrette. <br></br>
Dai titoli originali è ancora presente l'elemento caratteristico "(original title)" che viene sostituito con una stringa vuota una volta messo come valore nel dizionario. Nel caso in cui si trovassero chiavi duplicate il codice avviserebbe con un print.

In [3]:
diz_titles = {}
for i in range(len(movies)):
    if movies["Translated"].iloc[i] not in diz_titles.keys():
        diz_titles[movies["Translated"].iloc[i]] = str(movies["Original"].iloc[i]).replace("(original title)", "")
    else:
        print("Duplicate!")

<font size=3>

Sono state provati diversi metodi per far corrispondere i titoli geolocalizzati con quelli originali. È stata creata una nuova variabile e inizializzata con valori nulli per tutte le righe, successivamente sono stati applicati diversi metodi:
* Il metodo di iterazione tramite `.iloc[]` in cui un indice viene fatto scorrere fra tutte le righe del dataset. Questa soluzione è pratica, e sarà utilizzata anche più avanti, quando si devono fare dei confronti o individuare degli elementi all'interno delle righe selezionate, ma non è ottimizzata per la scrittura di valori.
* Il metodo `.iterrows()` utilizza la stessa logica ma per inserire i nuovi valori si applica un'ulteriore funzione `.set_value`. La combinazione dei due risulta essere migliore del metodo precedente.
* La funzione `.map()` è specifica per l'applicazione iterativa di una funzione, in questo caso trovare le corrispondenze con le chiavi di un dizionario.

Con questo ultimo metodo si passa dalle diverse ore stimate con `.iloc` e dalla mezz'ora con `iterrows()` a molto meno di un secondo totale. Prima di effettuare questo assegnamento viene modificata un'opzione di `pandas` per evitare che vengano stampati a video dei warning in quanto di default la libreria stampa un warning quando si sostituiscono valori all'interno di un dataframe.

In [4]:
%%time
imdb_df["Original"] = ""
pd.options.mode.chained_assignment = None
imdb_df["Original"] = imdb_df["Title"].map(diz_titles)

Wall time: 68.8 ms


<br></br>
## [1.2 Recensioni Float](#1.2)

<font size=3>

Questo blocco di codice è stato inserito in seguito a degli errori che sono apparsi più avanti nelle analisi. Nonostante la funzione di scraping `reviews` avesse delle specifiche sia per quando il testo della recensione fosse stato individuato sia per il caso contrario, alcuni valori risultano essere vuoti. Questi pochi errori sono probabilmente dovuti al fatto che la recensione è stata individuata ma non salvata correttamente. <br></br>
Il contenuto di queste celle è stato salvato come un oggetto `float` invece che stringa e l'applicazione della funzione `len()` in seguito porta ad un errore in quanto, a differenza delle stringhe, su Python gli oggetti float non hanno lunghezza. <br></br>

Per sistemare queste eccezioni si controlla se la tipologia del commento è float, in caso affermativo la cella viene trasformata in una stringa vuota. In questo caso il metodo `iloc[]` funziona rapidamente in quanto si lavora su poche osservazioni e il valore da sostituire è predefinito. Non c'è differenza di velocità fra questo approccio e quello con `iterrows()`.

In [5]:
%%time
for i in range(len(imdb_df)):
    if type(imdb_df["Review"].iloc[i]) == float:
         imdb_df["Review"].iloc[i] = ""

Wall time: 7.76 s


<br></br>
## [1.3 Pre-Processing Recensioni](#1.3)

<font size=3>

La seguente funzione ha il compito di eliminare alcuni tag presenti nei commenti, nello specifico i tag `\r` e `\n`. Questi tag sono presenti in coppia `\r\n` anche se la loro funzionalità singola è la stessa, ovvero quella di creare una nuova linea. Da una risposta su [stackoverflow](https://stackoverflow.com/questions/14606799/what-does-r-do-in-the-following-script) si apprende che quella che può sembrare una ridondanza è invece richiesto dal protocollo [Telnet](https://tools.ietf.org/html/rfc854).

In [6]:
def remove_back_tags(string):
    string = string.replace("\r", "").replace("\n", "")
    return (string)

<font size=3>

La funzione `pre_processing` forza l'elemento in ingresso ad essere trattato come una lista e su di essa sono svolte delle operazioni di pre-processing. Le diverse funzioni sono prese dalla libreria `gensim` più nello specifico dal modulo `gensim.parsing`:
* `strip_multiple_whitespaces`: nel caso ci siano più spazi bianchi consecutivi questi vengono condensati in uno solo.
* `strip_numeric`: rimuove i caratteri numerici da una stringa, funziona sia nel caso di numeri considerati stringhe, sia da termini alfanumerici.
* `strip_tags`: rimuove le tag in codice HTML e altri tag come visti in precedenze. 
* `remove_back_tags`: la funzione definita sopra, è necessaria in quanto non sempre la funzione `strip_tags` identifica correttamente quella coppia di tag.

La lista `my_filter` è creata con le funzioni di pre-processing al suo interno; questa è passata come parametro di un'altra funzione, `preprocess_string`, sempre della libreria gensim. Questa funzione è quella di default che effettua il pre-processing nella quale è possibile lasciare le funzioni di base o passare una lista con delle funzioni specifiche come in questo caso. <br></br>
Dato che l'oggetto in ingresso viene forzato ad essere una lista si applica lo slicing per accedere a tutti gli elementi uno alla volta, e per ogni termine all'interno di essi viene applicato il pre-processing. Per velocizzare tutte le operazioni viene applicata una list comprehension che restituisce come oggetto proprio una lista. <br></br>
Poiché si lavora a livello di singola parola, alla fine delle modifiche si ricrea una stringa corrispondente al testo totale della recensione tramite la funzione `join()`.

In [8]:
def pre_processing(texts: list):
    my_filter = [
        gensim.parsing.strip_multiple_whitespaces,
        gensim.parsing.strip_numeric,
        gensim.parsing.strip_tags,
        remove_back_tags
    ]

    texts = [preprocess_string(x, filters=my_filter) for x in (tqdm(texts[0:len(texts)]))]
    texts = [" ".join((x)) for x in texts if type(x) == list]

    return texts

<font size=3>

La colonna delle recensioni è passata in input alla funzione `pre-processing` e salvata nell'oggetto `text`. Successivamente si elimina dal dataset la vecchia colonna delle recensioni e la si sotituisce con il nuovo output.

In [9]:
text = pre_processing(imdb_df["Review"])

imdb_df.drop("Review", axis=1, inplace=True)
imdb_df["Review"] = text

100%|████████████████████████████████████████████████████████████████████████| 784259/784259 [01:53<00:00, 6890.97it/s]


<br></br>
## [1.4 Gestione Forme Contratte](#1.4)

<font size=3>

Un problema del pre-processing sorge quando si deve far fronte alle forme contratte. Tramite le funzioni di rimozione della punteggiatura e caratteri speciali (non ancora utilizzate) le forme contratte avrebbero tutti i caratteri uniti o in altri casi si andrebbero a creare due parole distinte. <br></br> Un ulteriore complicazione sorge dal fatto che la stessa parola può essere rappresentata in due modi differenti, ad esempio `it's` e `it\'s` sono due forme contratte che compaiono frequentemente nelle recensioni. <br></br>

La gestiore di queste parole è fondamentale in quanto uno degli step successivi prevede l'identificazione dei nomi e le parole non riconosciute dal POS-tagger sono automaticamente classificate come nomi. In caso di separazione delle forme contratte, ci sarebbero delle parole non esistenti e dunque identificate come nomi. In aggiunta le forme negative, oltre a non essere riconosciute come parole nel dizionario, perdono anche la loro valenza negativa stravolgendo il sentiment della frase. <br></br>

Per sistemare queste forme contratte e scriverle per esteso è stato creato un dizionario dove la chiave è la forma contratta e il valore il numero di volte in cui essa compare. L'algoritmo itera fra tutte le recensioni e con la funzione `split()` crea una lista di parole per ognuna di esse. È necessario trasformare ogni recensione in una lista dato che ogni elemento verrà trattato come una parola intera mentre passando direttamente una stringa l'algoritmo prenderebbe un carattere alla volta. Si controlla se nella parola è presente una contrazione e in caso positivo la si aggiunge al dizionario o si aumenta il conteggio già presente.

In [15]:
diz_end = {}
for i in tqdm(range(len(imdb_df))):
    word_list = imdb_df["Review"].iloc[i].split(" ")
    for word in word_list:
        if "\'" in word:
            if word in diz_end:
                diz_end[word] = diz_end[word] + 1
            else:
                diz_end[word] = 1

100%|███████████████████████████████████████████████████████████████████████| 784259/784259 [00:27<00:00, 28281.71it/s]


<font size=3>
    
Con la funzione `Counter` si estraggono le occorrenze più frequenti.

In [16]:
Counter(diz_end).most_common(10)

[("it's", 320913),
 ("don't", 221143),
 ("It's", 198018),
 ("didn't", 147291),
 ("doesn't", 138952),
 ("I'm", 128050),
 ("can't", 93444),
 ("isn't", 87013),
 ("I've", 83683),
 ("that's", 75800)]

<font size=3>
    
In base ai risultati ottenuti si crea un dizionario in cui le chiavi sono le forme contratte e i valori quelle estese. Sono state considerate solamente i casi con maggiore frequenza e soprattutto quelli in cui fosse presente una negazione. Ci sono tantissime altre occorrenze non considerate, ad esempio il genitivo sassone, ma queste verranno in parte sistemate successivamente durante il pre-processing.

In [17]:
abbr_diz = {"it's":"it is", "don't":"do not", "It's":"it is", "didn't":"did not", "doesn't":"does not", "I'm":"I am",
            "can't":"can not", "isn't":"is not", "I've":"I have", "that's":"that is", "wasn't":"was not", "he's": "he is",
            "you're":"you are", "there's":"there is", "couldn't":"could not", "won't":"will not", "film's":"film is",
            "I'd":"I would", "wouldn't":"would not", "There's":"there is", "you'll":"you will", "she's":"she is",
            "Don't":"do not", "haven't":"have not", "That's":"that is", "they're":"they are", "I'll":"I will", 
            "aren't":"are not", "He's":"he is", "who's":"who is", "what's":"what is", "you've":"you have", 
            "movie's":"movie is", "we're":"we are", "weren't":"were not", "character's":"character is", "hasn't":"has not",
            "shouldn't":"should not", "She's":"she is", "you'd":"you would", "could've":"could have", "we've":"we have",
            "would've":"would have", "hadn't":"had not", "What's":"What is", "they've":"they have", "i'm":"I am",
            "You'll":"You will", "They're":"They are", "i've":"I have", "Can't":"Can not", "he'd":"he would",
            "ain't":"is not", "Here's":"Here is", "didn't":"did not", "We're":"We are", "You're":"You are", 
            "he'll":"he will", "they'd":"they would", "isn't.":"is not.", "should've":"should have", "here's":"here is",
            "I\'m'":"I am", "we'll":"we will", "they'll":"they will", "it'll":"it will", "(I'm":"I am", 
            "don't.":"do not.", "Didn't":"Did not", "It\'s'":"It is", "wasn't.":"was not.", "doesn't.":"does not.",
            "We've":"We have", "we'd":"we would", "You've":"You have", "(it's":"it is", "else's":"else is", 
            "Doesn't":"Does not", "Isn't":"Is not", "i'd":"I would", "she'd":"she would", "DON'T":"do not", 
            "You'd":"You would", "she'll":"she will", "who've":"who have", "Don\'t":"do not", "don't,":"do not,",
            "it'd":"it would", "Wouldn't":"Would not", "Couldn't":"Could not", "i'll":"I will", "can't.":"can not.",
            "Wasn't":"was not", "(that's":"that is", "its'":"its", "didn't,":"did not,", "it\'s":"it is", 
            "that'll":"that will", "(don't'":"do not", "isn't,":"is not,", "won't.":"will not.", "doesn't,":"does not",
            "Haven't":"Have not", "don\'t":"do not", "wasn't,":"was not", "aren't.":"are not", "can't,":"can not"}

<br></br>
<br></br>
# [2. Frasi](#2.)
<br></br>
## [2.1 Passaggio da Recensioni a Frasi](#2.1)

<font size=3>
    
Come letto in alcuni paper, una delle tecniche per iniziare una aspect-based sentiment analysis è quella di dividere l'intera recensioni in singole frasi. I caratteri delimitatori presi in considerazione sono `?` e `.`, nel caso in questione sono stati considerati anche i punti esclamativi. <br></br>
Nelle seguenti righe viene creata una nuova colonna vuota in cui sono inserite, come liste, le frasi di ogni recensione. In ogni recensione i punti di domanda e i punti esclamativi sono trasfomati in punti che sono poi utilizzati come delimitatore per la funzione `split()`. <br></br>

In altri parti del codice si utilizzeranno altre librerie o diverse funzioni per sistemare delle componenti testuali. In questo caso concatenare più funzioni `replace()` risulta essere [il metodo più veloce](https://stackoverflow.com/questions/3411771/best-way-to-replace-multiple-characters-in-a-string).

In [18]:
imdb_df["Sentences"] = ""
for index, row in tqdm(imdb_df.iterrows()):
    imdb_df.set_value(index, "Sentences", row["Review"].replace("?", ".").replace("!", ".").split("."))

0it [00:00, ?it/s]C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
784259it [01:02, 12531.50it/s]


<font size=3>

Sempre nei paper è specificato come le frasi duplicate all'interno della stessa recensione debbano essere rimosse. Questo deriva dal fatto che spesso gli utenti per sottolineare ed enfatizzare un concetto possono ripetere la stessa frase più volte influenzando il sentiment globale. Questo passaggio serve anche per eliminare quelle frasi che hanno meno di due caratteri, in sostanza i casi in cui, in presenza di più punti consecutivi, si fossero create delle liste vuote o con una sola lettera. <br></br>

Per trovare gli elementi singoli di una lista si utilizza il metodo `set()`, questa funzione può essere applicata direttamente alla lista, ma ha lo svantaggio di non mantenere l'ordine degli elementi, il risultato sarebbero frasi all'interno di una recensione in ordine sparso. Per aggirare questo problema si itera per ogni recensione con `iterrows()` e si inizializza ogni volta un oggetto set vuoto. <br></br>
Si crea una list comprehension in cui sono inseriti solamente gli oggetti che non sono presenti nel set inizializzato precedentemente e che hanno lunghezza maggiore di uno. Il risultato è sempre una lista ma con elementi non duplicati. Per velocizzare il processo viene creato un assegnamento locale tramite `seen_add = seen.add` e su questo si effettua il check di presenza o assenza.

In [19]:
imdb_df["Sentences2"] = ""
for index, row in tqdm(imdb_df.iterrows()):
    seen = set()
    seen_add = seen.add
    imdb_df.set_value(index, "Sentences2", [x for x in row["Sentences"] if not (x in seen or seen_add(x)) and len(x) > 1])

0it [00:00, ?it/s]C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
784259it [01:11, 11034.93it/s]


<br></br>
## [2.2 Sostituzione delle Forme Contratte](#2.2)

<font size=3>

Prima di procedere con un secondo step di pre-processing, avendo eliminato i duplicati all'interno delle recensioni si passa a sostituire le forme contratte. La logica dietro a questo blocco di codice è quella vista in altre parti, ma è leggermente più articolata in quanto si deve ragionare con più elementi all'interno di una singola recensione:
* L'iterazione per ogni recensione è effettuata sempre mediante `iterrows()`, per ogni recensione viene creata una lista, `new_review`.
* Si itera per ogni frase all'interno della recensione e si crea una lista in cui ogni elemento è una parola di una frase. Si crea un'uleriore lista, `new_sent`, in cui verrà ricreata la singola frase.
* Per ogni parola nella lista della frase si controlla se l'elemento è presente nelle chiavi del dizionario delle abbreviazioni. In caso affermativo nella lista `new_sent` viene inserito il valore corrispondente alla chiave, altrimenti la parola passata in input.
* A questo punto `new_sent` contiene le singole parole di una frase, per ritornare ad un'unica frase si applica la funzione `join()` e la stringa risultate viene salvata in `new_review`. Si procede per tutte le frasi della recensione, al termine la lista `new_reviews` conterrà tanti elementi quante erano le frasi originarie della recensione.

Finita l'analisi di una recensione, l'elemento `new_rewiew` è salvato in una nuova variabile e si procede per via iterativa.

In [26]:
imdb_df["Sentences3"] = ""
for index, row in tqdm(imdb_df.iterrows()):
    new_review = []
    for sentence in row["Sentences2"]:
        temp_sent = sentence.split(" ")
        new_sent = []
        for word in temp_sent:
            if word in abbr_diz:
                new_sent.append(abbr_diz[word])
            else:
                new_sent.append(word)
        new_review.append(" ".join(new_sent))
    imdb_df.set_value(index, "Sentences3", new_review)

0it [00:00, ?it/s]C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
784259it [01:58, 6640.01it/s]


<br></br>
## [2.3 Pre-Processing](#2.3)

<font size=3>

Per completare il pre-processing è necessario creare un'altra funzione, `reduce_length`, per sistemare alcuni casi di caratteri ripetuti. Non è raro trovare alcuni termini scritti con tante lettere ripetute per sottolinearne la valenza. La seguente funzione rimuove $3$ o più caratteri consecutivi e li riporta a $2$:
* Attraverso la libreria `re` e la funzione `compile` si inizializzano i pattern da identificare. L'espressione regolare è composta dai seguenti elementi:
    * `(.)`: indica che tutti i caratteri saranno presi in considerazione.
    * `\1`: indica un raggruppamento di ordine $1$, in questo caso è ovvio poiché alla funzione sarà passato un termine alla volta.
    * `{2,}`: individua qualsiasi carattere ripetuto $2$ o più volte.
* Tramite la funzione `sub` i pattern individuati sono sostituiti, il numero di `\1` indica il numero di caratteri da mettere come sostituzione, in questo caso $2$.
<br></br>

Come è intuibile questa correzione non restituisce delle parole necessariamente corrtte, ad esempio nel caso in cui si trovino tre lettere ma la parola corretta ne contiene solo una, da questa funzione si avrà in output una parola errata con una doppia. Alcune delle parole potrebbero essere sistemate immediatamente, ma ridurre il numero di caratteri consecutivi aiuta notevolmente gli algoritmi di spelling check che saranno implementati successivamente in quanto la distanza dal termine originale sarà resa minore.

In [27]:
def reduce_length(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

<font size=3>

La seguente funzione, `pre-processing2`, ha lo stesso comportamento della prima: si trasforma l'elemento in input in una stringa e si applicano diverse funzioni di pre-processing. La differenza principale rispetto a prima è che adesso le recensioni sono delle liste con un insieme di frasi, risulta quindi necessario applicare un doppio ciclo for nella list comprehension in modo da scorrere prima per tutte le recensioni e per tutte le frasi nelle recensioni. <br></br>

Le funzioni di pre-processing applicate in quest step sono:
* `strip_multiple_whitespaces`: già usata in precedenza, rimuove spazi bianchi multipli fra parole. Non dovrebbero essercene ma la si utilizza per sicurezza.
* `strip_punctuation`: rimuove i caratteri di punteggiatura. Non è stata usata in precedenza per i motivi già esposti.
* `strip_non_alphanum`: rimuove i caratteri speciali, anche all'interno di una stringa.
* `reduce_length`: rimuove un carattere ripetuto più di $2$ volte consecutivamente.

In [28]:
def pre_processing2(texts: list):
    my_filter = [
        gensim.parsing.strip_multiple_whitespaces,
        gensim.parsing.strip_punctuation,
        gensim.parsing.strip_non_alphanum,
        reduce_length
    ]

    texts = [preprocess_string(x, filters=my_filter) for unique in tqdm((texts[0:len(texts)])) for x in unique]
    texts = [" ".join((x)) for x in texts]

    return texts

<font size=3>

Si crea una variabile che raccoglie l'output della funzione. Nel passaggio precedente la colonna del dataset era eliminata e sostituita con la nuova variabile creata, in questo caso non è possibile farlo in quanto la lunghezza dei due elementi non è la stessa. L'output della funzione `pre_processing2` è un'unica lista in cui sono presenti tutte le frasi di ogni singola recensione e non una lista di liste.

In [29]:
text = pre_processing2(imdb_df["Sentences3"])

100%|████████████████████████████████████████████████████████████████████████| 784259/784259 [04:33<00:00, 2871.01it/s]


<font size=3>

La soluzione di ritornare ad una lista unica può sembrare non molto pratica, però ha sicuramente dei vantaggi nella semplicità del codice ed è possibile con poche righe ricreare liste di liste con le stringhe pre-processate. <br></br>

Sempre attraverso un ciclo con `iterrows()` viene realizzato questo processo: 
* Fuori dal ciclo si inizializza un contatore.
* Per ogni recensione non pre-processata si conta quante siano le frasi al suo interno
* Attraverso uno slicing fra il contatore e la lunghezza della recensione si individuano gli elementi corrispondenti nella lista pre-processata.

In [30]:
imdb_df["Sentences4"] = ""
cont = 0
for index, row in tqdm(imdb_df.iterrows()):
    l = len(row["Sentences3"])
    imdb_df.set_value(index, "Sentences4", text[cont:(cont+l)])
    cont = cont + l

0it [00:00, ?it/s]C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
784259it [01:04, 12130.38it/s]


<font size=3>

Per evitare di riprodurre tutto il codice ogni volta che viene attivato il kernel, si salva il dataframe con `pickle` e lo si ricaricherà ad ogni nuova sessione. Dal file sono eliminate le variabili con i passaggi intermedi di pre-processing sulle recensioni, lasciando la recensione originale e il risultato finale.

In [31]:
imdb_df_copy = imdb_df
del imdb_df_copy["Sentences"]
del imdb_df_copy["Sentences2"]
del imdb_df_copy["Sentences3"]
with open('imdb_df4', 'wb') as fp:
    pickle.dump(imdb_df_copy, fp)

In [3]:
with open('imdb_df4', "rb") as input_file:
    imdb_df = pickle.load(input_file)

<br></br>
<br></br>
# [3. Spelling Correction](#3)
<br></br>
## [3.1 Nomi](#3.1)

<font size=3>
    
Per effettuare una correzione degli errori di ortografia si devono prima di tutto individuare i termini che sono stati scritti erroneamente. Il compito è facile quando una parola è scritta in maniera errata e si è creato un termine che non esiste in lingua inglese, più complicato quando un errore di battitura crea una parola reale. <br></br>
Dovendo lavorare con delle recensioni di film, si troveranno un elevato numero di riferimenti agli attori e alle persone coinvolte nella realizzazione del film. I nomi e cognomi degli attori possono facilmente essere riconosciute come parole errate, anche se non dovrebbero esserlo. Grazie alla lista di attori scaricata in precedenza è possibile filtrare queste parole in modo tale da non analizzarle.

In [22]:
info.head()

,Unnamed: 0,Title,Director,Actor,Fict_actor,Writer
0,0,The Irishman (2019),['Martin Scorsese'],"['Robert De Niro', 'Al Pacino', 'Joe Pesci', '...","['Frank Sheeran', 'Jimmy Hoffa', 'Russell Bufa...","['Charles Brandt', 'Steven Zaillian']"
1,1,Cena con delitto - Knives Out (2019),['Rian Johnson'],"['Daniel Craig', 'Chris Evans', 'Ana de Armas'...","['Benoit Blanc', 'Ransom Drysdale', 'Marta Cab...",['Rian Johnson']
2,2,C'era una volta a... Hollywood (2019),['Quentin Tarantino'],"['Leonardo DiCaprio', 'Brad Pitt', 'Margot Rob...","['Rick Dalton', 'Cliff Booth', 'Sharon Tate', ...",['Quentin Tarantino']
3,3,Joker (2019),['Todd Phillips'],"['Joaquin Phoenix', 'Robert De Niro', 'Zazie B...","['Arthur Fleck', 'Murray Franklin', 'Sophie Du...","['Todd Phillips', 'Scott Silver', 'Bob Kane', ..."
4,4,Ad Astra (2019),['James Gray'],"['Brad Pitt', 'Tommy Lee Jones', 'Ruth Negga',...","['Roy McBride', 'H. Clifford McBride', 'Helen ...","['James Gray', 'Ethan Gross']"


<font size=3>

Quando è stato salvato il dataframe in un `.csv` le liste all'interno delle colonne sono state automaticamente trasformate in delle stringhe poiché in questo formato non è possibile salvare alcune strutture di Python come le liste. Gli elementi, nonostante abbiano la corretta formattazione di una lista, sono in realtà letti come delle stringhe. <br></br>
La funzione `flat_list` che segue risolve questo problema e ritorna una lista vera e propria. Gli step della funzione sono:
* Si sostituiscono con degli spazi nulli gli elementi caratteristici delle liste che in realtà sono presenti nella stringa, ovvero `[`, `]` e `'`.
* Se è presente `,` nella nuova stringa significa che originariamente la lista aveva più elementi, con la funzione `split()` si crea una lista di tanti elementi quanti sono i nomi presenti.
* Volta per volta i nomi sono inseriti nella lista. Per la spelling correction serve semplicemente una lista di nomi, per adesso non è necessario mantenere le relazioni fra persone e film.

Fra i valori in ingresso della funzione è presente anche un parametro opzionale, `unique`, dove se ha valore $1$ l'elemento restituito è una lista degli elementi unici trovati in tutta la colonna, altrimenti mette in output la lista senza togliere i duplicati.

In [32]:
def flat_list(df_column, unique=1):
    new_list = []
    for element in df_column:
        element = element.replace("[", "").replace("]", "").replace("'", "")
        if "," in element:
            name_list = element.split(",")
            for name in name_list:
                new_list.append(name)
        else:
            new_list.append(element)

        if unique == 1:
            new_list = list(set(new_list))
    
    return (new_list)

In [33]:
director = flat_list(info["Director"])
actor = flat_list(info["Actor"])
fict_actor = flat_list(info["Fict_actor"])
writer = flat_list(info["Writer"])

info_list = [director, actor, fict_actor, writer]

<br></br>
## [3.2 Dizionario](#3.2)

<font size=3>
    
Per identificare le parole che esistono realmente, una delle possibili soluzioni è quella di prendere un dizionario esterno e controllare di volta in volta se la parola nella recensione sia nel dizionario o meno. <br></br>
Esistono diversi dizionari disponibili, il più completo è forse quello di `StanfordNLP`, una libreria che contiene una pipeline per Natural Language Processing, con diversi step che possono essere implementati anche in questo progetto. 
<br></br>

Ci sono state inizialmente delle difficoltà ad installare questa libreria per via di un problema legato a `PyTorch`, una libreria essenziale per installare alcuni pacchetti. Il sito del pacchetto forniva una finestra interattiva con la quale veniva visualizzato il codice da inserire per installare la libreria a seconda della versione di Python ed altre componenti. Per tutte le combinazioni si presentava sempre un problema nell'ambiente non risolvibile. Per installare correttamente la libreria è stato necessario aggiornare Anaconda alla versione $3.7$.
<br></br>

Oltre a quello di Stanford, il dizionario più completo reperibile è quello presente su [questo GitHub](https://github.com/dwyl/english-words), un dizionario in lingua inglese con $460'000$ parole. La versione scaricata è quella light in quanto comprende solamente parole senza l'aggiunta di caratteri numerici o speciali che sono già stati rimossi nel pre-processing. Vista la completezza di questo file si è deciso di usare questo come dizionario e `StanfordNLP` per il resto delle analisi.<br></br>

Il file salvato nella directory viene letto riga per riga e ogni parola, tolto `\n` indicatore di nuova riga, viene aggiunta ad un dizionario. La creazione di un dizionario al posto di una lista è per velocizzare il controllo delle parole che avviene al passo successivo.

In [34]:
%%time
word_diz = {}
with open("words_alpha.txt") as w:
    for line in w:
        word_diz[line.strip("\n")] = 1

Wall time: 184 ms


<font size=3>
    
Al fine di arricchire il dizionario sono inseriti gli elementi presenti nelle informazioni dei film.
* È stata creata in precedenza una lista con dentro le singole liste di attori e delle altre informazioni.
* Si passa ogni elemento di ogni lista. Se l'elemento ha uno spazio, nome e cognome, viene spezzato e l'oggetto viene aggiunto al dizionario se non è già presente. Si aggiunge direttamente l'elemento senza spezzarlo se non è presente alcuno spazio.
* Gli elementi sono aggiunti sia nella forma minuscola che mantenendo la maiuscola all'inizio. Questa scelta, che aumenta la dimensione del dizionario, è necessaria per i seguenti motivi:
    * Non sempre nelle recensioni i nomi sono scritti con la maiuscola. La maggior parte degli utenti è attenta a scrivere correttamente ma tanti altri non lo sono.
    * Il modello di `StanfordNLP` non sempre riconosce le forme tutte in minuscolo.
<br></br>

In questo caso il valore assegnato alla chiave è sempre unitario.

In [35]:
for info in info_list:
    for element in info:
        if " " in element:
            temp = element.split(" ")
            for token in temp:
                if token.lower() not in word_diz.keys():
                    word_diz[token.lower()] = 1
                    word_diz[token] = 1
        else:
            if element.lower() not in word_diz.keys():
                word_diz[element.lower()] = 1
                word_diz[element] = 1

<font size=3>
    
Sicuramente la libreria `StanfordNLP` è più completa, ma dal profilo prestazionale richiede più tempo rispetto ad altre. L'idea è quindi di fare una prima analisi con il dizionario appena letto e controllare successivamente l'insieme ridotto di parole che non sono state riconosciute. 

Le parole che non esistono nel dizionario sono dei possibili candidati ad essere dei termini scritti erroneamente o dei NER.

In [36]:
%%time
unknown_words = {}
for i in tqdm(range(len(imdb_df))):
    for sentence in imdb_df["Sentences4"].iloc[i]:
        temp = sentence.split(" ")
        for word in temp:
            if word.lower() not in word_diz.keys():
                if word in unknown_words.keys():
                    unknown_words[word] += 1
                else:
                    unknown_words[word] = 1

100%|███████████████████████████████████████████████████████████████████████| 784259/784259 [01:05<00:00, 12024.20it/s]


Wall time: 1min 5s


<br></br>
## [3.3 NER](#3.3)

<font size=3>

Sono caricati i modelli in lingua inglese dalla libreria `StanfordNLP`.

In [28]:
stanfordnlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: C:\Users\Hp\stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: C:\Users\Hp\stanfordnlp_resources\en_ewt_models.zip


100%|████████████████████████████████████████████████████████████████████████████████| 235M/235M [04:04<00:00, 962kB/s]



Download complete.  Models saved to: C:\Users\Hp\stanfordnlp_resources\en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


<font size=3>
 
In un primo momento la fase di NER è stata effettuata importando la libreria `StanfordNERTagger` dal paccketto `nltk`. Questa via prevedeva la creazione di un tagger che prendeva in input i seguenti elementi:
* Un modello già addestrato
* Il file tagger presente nello .zip scaricato da StanfordNLP
* La codifica

Il file `english.all.3class.distsim.crf.ser.gz` non è altro che un file compresso al cui interno è presente come unico elemento il modello già addestrato. Il secondo valore in ingresso è `stanford-ner.jar`, il file Java che permette di connettersi al modello e di identificare le eventuali entità all'interno del testo che verrà inserito. Per la codifica è stato scelto `utf-8`, che è già quello di default, e permette di leggere correttamente anche eventuali caratteri stranieri. <br></br>

Come ultimo passaggio è stato necessario specificare la directory in cui è installato Java.

In [83]:
st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz', 'stanford-ner.jar', encoding='utf-8') 

java_path = "C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath\\java.exe"
os.environ['JAVAHOME'] = java_path

<font size=3>
    
In maniera molto semplice si scorre fra tutte le chiavi del dizionario e si applica la funzione definita sopra. Per sicurezza la parola viene trasformata in una stringa e applicato il metodo `split()` per renderla una lista, altrimenti la funzione interpreterebbe ogni singolo carattere dell'input come una parola. <br></br>
L'output viene aggiunto alla lista. Il formato dell'output è una lista al cui interno sono contenute una serie di tuple, tante quante le parole date in ingresso, in questo caso passando una parola alla volta c'è una tupla sola:
* Il primo elemento è la parola in ingresso.
* Il secondo elemento della tupla è l'entità identificata dal modello.

In [122]:
ner = []
for word in tqdm(unknown_words.keys()):
    tag = st.tag(str(word).split())
    ner.append(tag)

<font size=3>

L'algoritmo funziona correttamente, ma soffre di estrema lentezza computazionale: su questa macchina la velocità media era di $2$&ndash;$3$ parole al secondo, con un tempo stimato totale di oltre $100$ ore.

<br></br>
### [3.3.1 tqdm Output](#3.3.1)

<font size=3>

La libreria `tqdm` è spesso utilizzata in questo notebook perché permette di calcolare il tempo di esecuzione di una funzione che itera su un oggetto di lunghezza definita. L'output dovrebbe essere una barra che si completa al passare delle iterazioni. <br></br>
Se un blocco di codice però viene interrotto e quindi la barra non raggiunge la fine, quando viene richiamata la funzione, anche in un altro blocco di codice, è possibile che invece di visualizzare una sola riga, la barra di completamento sia stampata a ripetizione. Oltre ad essere un problema grafico, il print continuo di nuovi elementi rallenta leggermente il codice. Il primo metodo che richiedeva $100$ è stato bloccato e questo ha creato il problema nella funzione.
<br></br>
Importare nuovamente la libreria non ha effetto, ma, come evidenziato in [questa discussione su Github](https://github.com/tqdm/tqdm/issues/375) è possibile sistemare il problema:
* La prima soluzione è usare la funzione già presente nel pacchetto, `tqdm._instances.clear()`, con il quale viene ripulita la "memoria" della funzione delle precedenti operazioni eseguite. Funziona parzialmente in quanto alcune volte vengono stampate ancora delle nuove linee.
* Il secondo metodo è ridefinire la funzione eliminando la memoria di quello passato attraverso un nuovo metodo non presente nel pacchetto originale. Anche in questo caso la memoria non tutte le volte viene ripulita correttamente.
* Il terzo metodo consiste nel chiamare la funzione del punto precedente ed importare nuovamente la libreria. Dato che la funzione precedente ha di fatto sovrascritto il nome, questa volta importare la libreria è come se lo si facesse per la prima volta e quindi risolve il problema.

In [274]:
tqdm._instances.clear()

In [275]:
def tqdm(*args, **kwargs):
    from tqdm import tqdm as tqdm_base
    if hasattr(tqdm_base, '_instances'):
        for instance in list(tqdm_base._instances):
            tqdm_base._decr_instances(instance)
    return tqdm_base(*args, **kwargs)

In [276]:
from tqdm import tqdm

<br></br>
### [3.3.2 NLP Server](#3.3.2)

<font size=3>
  
Fortunatamente è possibile velocizzare notevolmente i tempi computazionali, come riportato in [questa discussione su Stackoverflow](https://stackoverflow.com/questions/57424885/how-to-speedup-stanford-nlp-in-python) è possibile stabilire una connessione con il `Stanford NLP Server` che risulta essere la soluzione ottimale per svolgere un NER. <br></br>
L'operazione richiede due passaggi:
* Il primo è da effettuarsi sul prompt dei comandi. Selezionata la directory in cui è stato estratto lo .zip scaricato dal sito di Stanford si immette il seguente codice `java -Djava.ext.dirs=./lib -cp stanford-ner.jar edu.stanford.nlp.ie.NERServer -port 9199 -loadClassifier ./classifiers/english.all.3class.distsim.crf.ser.gz`. In questo modo si accede al file `stanford-ner.jar` che è lo stesso specificato nel metodo precedente, nella parte successiva si chiama la connessione al NERServer attraverso una porta ed infine si carica lo .zip del modello, anche in questo caso è lo stesso specificato nel primo metodo.  
* Se i file sono stati trovati, il prompt li caricherà e si potrà chiamare la libreria `Ner` che contiene l'omonima classe che permette di fare entity recognition collegandosi ad un server esterno.

In [37]:
tagger = Ner(host='localhost', port=9199)

<font size=3>

Il codice è simile al metodo precedente, in questo caso viene utilizzata la funzione `get_entities` per richiamare l'output del NER. Anche in questo caso la struttura dell'output è la stessa: una lista con dentro tante tuple quante sono le parole in ingresso.

In [38]:
ner_words = []
for word in tqdm(unknown_words.keys()):
    ner_words.append(tagger.get_entities(word.split()[0]))

100%|████████████████████████████████████████████████████████████████████████| 219420/219420 [02:32<00:00, 1436.86it/s]


<font size=3>

La connessione con il server riduce drasticamente i tempi di computazione che ritornano ad essere accettabili. Classificate le entità, si scorre all'interno di ogni elemento, si divide la tupla e si prende l'entità identificata. Si effettua un conteggio per valutare le frequenze delle entità. <br></br>
L'entità `O` indica una parola che non è stata riconosciuta come nessuna entità, e sono anche le più numerose. L'altra metà sono risconosciute come delle persone mentre hanno una frequenza molto inferiore i luoghi e le organizzazioni.

In [39]:
idx = []
for entity in ner_words:
    temp = str(entity[0]).split(",")
    idx.append(temp[1])
Counter(idx).most_common()

[(" 'O')", 114246),
 (" 'PERSON')", 101644),
 (" 'LOCATION')", 2372),
 (" 'ORGANIZATION')", 1158)]

<font size=3>

Analizzando anche solo i primi $100$ risultati si notano diversi errori di identificazione:
* Entità come `Porsche` o `Facebook` non sono riconosciute.
* `Nascar` è classificata come persona anche se dovrebbe rientrare sotto organizzazione.
* La parola `Motorport` è proprio un errore di spelling ma è classificata come persona.
* Alcune sigle come `DVD` non sono riconosciute.

In [40]:
temp_str = ''
for ner in ner_words[0:100]:
    temp_str += str(ner)[1:-1] + ' '
print(temp_str)

('CGI', 'O') ('Imax', 'O') ('tifoso', 'O') ('sportcars', 'O') ('Porsche', 'O') ('Corvair', 'PERSON') ('Chevelle', 'PERSON') ('racecar', 'O') ('Nascar', 'PERSON') ('Surtees', 'PERSON') ('autoracing', 'O') ('Biopics', 'O') ('goosebumps', 'O') ('IMAX', 'O') ('Phedon', 'O') ('Cineworld', 'PERSON') ('UK', 'LOCATION') ('minivans', 'O') ('SUVs', 'O') ('Papamichael', 'PERSON') ('McCusker', 'PERSON') ('Beltrami', 'PERSON') ('fearsomly', 'O') ('Facebook', 'ORGANIZATION') ('Audouy', 'PERSON') ('Orlandi', 'PERSON') ('RPMs', 'O') ('WW', 'O') ('racerturned', 'O') ('Brummie', 'PERSON') ('pulsequickening', 'O') ('screentime', 'O') ('motorsport', 'O') ('Petrolheads', 'O') ('skeeted', 'O') ('BGM', 'O') ('Eventhough', 'O') ('biopics', 'O') ('Iacocco', 'PERSON') ('bullxx', 'O') ('groundbreaking', 'O') ('alot', 'O') ('Caitrona', 'PERSON') ('autobody', 'O') ('Corroll', 'PERSON') ('Shellby', 'PERSON') ('Porsches', 'O') ('Ferraris', 'O') ('Broadley', 'PERSON') ('WTF', 'O') ('brummie', 'O') ('Magild', 'PERSON'

<br></br>
### [3.3.4 NER su Frasi](#3.3.4)

<font size=3>
    
Una possibile spiegazione può essere data dal fatto che i modelli sono allenati su delle frasi e non delle singole parole, è quindi possibile che in alcuni casi il modello non riconosce la parola ma ha un'alta confidenza che si tratti di una persona, o qualche altra entità, e la classifica di conseguenza. <br></br>

Partendo dalla lista di parole sconosciute si controlla frase per frase all'interno delle recensioni se almeno una di esse è presente. In caso affermativo tutta la frase viene passata alla funzione di NER e l'output viene salvato in una lista.

In [41]:
sentence_ner = []
for index, row in tqdm(imdb_df.iterrows()):
    for sentence in row["Sentences4"]:
        if any(word in unknown_words.keys() for word in sentence.split()):
            sentence_ner.append(tagger.get_entities(sentence))

784259it [34:35, 377.85it/s]


<font size=3>

Ottenere le entità dalle frasi richiede molto più tempo anche utilizzando la connessione del server, questa è anche una delle ragioni per cui è impraticabile usare questo metodo sull'interno dataset. <br></br>
Il risultato viene salvato e caricato per evitare di ripetere il procedimento ogni volta.

In [42]:
"""
with open('sentence_ner', 'wb') as fp:
    pickle.dump(sentence_ner, fp)
"""
    
with open('sentence_ner', "rb") as input_file:
    sentence_ner = pickle.load(input_file)

<font size=3>
    
Un modo veloce per controllare se la categorizzazione è la stessa è controllare la lunghezza della lista con gli elementi unici. Se una parola viene classificata come più entità diverse a seconda del contesto, la parola comparirà più volte e di conseguenza la lista sarà più lunga di quella precedente, che era di $219'420$ elementi come era visibile dall'output di `tqdm`. <br></br>

La funzione `reduce_length` ha permesso di scendere da 223 mila parole e di identificare un numero maggiore di luoghi e organizzazioni, rendendo la procedura più veloce ed accurata.

In [43]:
ner_words_sent = []
for element in tqdm(sentence_ner):
    for word in element:
        if word[0] in unknown_words.keys():
            ner_words_sent.append(word)
            
len(set(ner_words_sent))

100%|████████████████████████████████████████████████████████████████████| 1066180/1066180 [00:05<00:00, 185028.89it/s]


244215

<font size=3>

La categorizzazione attraverso le frasi restituisce risultati differenti rispetto a quella per singola parola. <br></br>
Dalle entità trovate sono create due liste, una in cui sono messi i termini e nella seconda l'entità relativa. Gli oggetti sono inseriti ad ogni iterazione per mantenere la corrispondenza degli indici.

In [44]:
ner_words_sent = []
ent_words_sent = []
for element in tqdm(sentence_ner):
    for word in element:
        if word[0] in unknown_words.keys():
            ner_words_sent.append(word[0])
            ent_words_sent.append(word[1])

100%|████████████████████████████████████████████████████████████████████| 1066180/1066180 [00:04<00:00, 213749.84it/s]


<font size=3>

Dalle due liste si crea un dataframe con parola ed entità relativa. Mediante `groupby` e `size()` si crea un nuovo dataframe con il conteggio di quante volte una parola è identificata come una certa identità. Si ordina infine il dataframe in modo descrescente per la nuova colonna di conteggio tramite la funzione `sort_values`.

In [45]:
unk_df = pd.DataFrame({"Word":ner_words_sent, "Entity":ent_words_sent})

unk_df = unk_df.groupby(["Word", "Entity"]).size().reset_index(name="count").sort_values(by=["count"], ascending=False)
unk_df.head(20)

,Word,Entity,count
32128,DVD,O,28818
63496,IMDb,O,17886
21167,CGI,O,17013
183503,filmmakers,O,13173
168374,cliché,O,12919
168382,clichés,O,9762
197632,kinda,O,8435
193601,indie,O,6611
17296,Bollywood,O,6600
168375,clichéd,O,6029


<font size=3>
    
Un dizionario è creato e prenderà come chiave la parola non riconosciuta e come valore l'entità più frequente ad essa associata. Questa soluzione è un'approssimazione in quanto non si ha la certezza che ad una parola sia attribuita l'entità corretta, però si dovrebbe avere una discreta accuratezza.    

In [46]:
entity_diz = {}
for index, row in tqdm(unk_df.iterrows()):
    if row["Word"] not in entity_diz.keys():
        entity_diz[row["Word"]] = row["Entity"]

244215it [00:19, 12590.04it/s]


<font size=3>
    
Se il più delle volte una parola viene riconosciuta come una persona, un'organizzazione o una località, difficilmente si tratta di un errore di battitura, ma di un'entità realmente esistente ma non presente nel dizionario caricato all'inizio. Seguendo questa logica si crea un dizionario uguale al precedente con l'unica differenza che vengono salvati solo quei termini non identificabili con nessuna entità specifica.

In [47]:
entity_diz_small = {}
for key in entity_diz.keys():
    if entity_diz[key] == "O":
        entity_diz_small[key] = entity_diz[key]

<br></br>
## [3.4 Librerie di Spelling](#3.4) 
<br></br>
<font size=3>
    
Sono state provate diverse librerie per la spelling correction. Di seguito sono riportati i risultati in termini computazionali delle analisi sulle prime $10$ recensioni. Le valutazioni saranno effettuate sia sulla base della velocità, sia sulla struttura dell'algoritmo implementato in quanto una delle note problematiche di questi algoritmi è la lunghezza computazionale. <br></br>

Per ottenere delle tempistiche più accurate, il caricamento delle librerie e le necessarie impostazioni da definire per utilizzare correttamente le funzioni sono effettuate in righe di codice separate.

<font size=3>
    
La prima libreria è `Word` presa dal pacchetto `textblob`. Il metodo prende in input una parola alla volta e prova a correggerla mettendo in output una lista di tuple con i possibili candidati e la loro probabilità. <br></br>
In [questa pagina](http://norvig.com/spell-correct.html) è presente la documentazione della strategia utilizzata. In sintesi la libreria ha un modello e un dizionario con delle parole e le rispettive frequenze, per ogni parola viene calcolata la probabilità che essa sia una parola nel dizionario. Questa probabilità che viene restituita in output è ponderata per la frequenza della parola nel dizionario e l'edit distance relativa. <br></br>

Secondo [questi](https://www.clips.uantwerpen.be/pages/pattern-en#spelling) risultati, citati anche nella pagina del pacchetto, la libreria dovrebbe avere un'accuratezza del $70\%$.

In [ ]:
from textblob import Word

In [121]:
%%time
for i in range(10):
    for sentence in imdb_df["Sentences4"].iloc[i]:
        temp = sentence.split()
        for word in temp:
            check = Word(word.lower())
            spelling = check.spellcheck()

Wall time: 11.6 s


In [ ]:
from pattern.en import suggest

<font size=3>
    
La [seconda libreria](https://rustyonrampage.github.io/text-mining/2017/11/28/spelling-correction-with-python-and-nltk.html) è `suggest` che lavora nello stesso identico modo di quella precedente. Anche se proviene da un paccheto diverso, la [documentazione](https://www.clips.uantwerpen.be/pages/pattern-en) rimanda alla stessa pagina di quello precedente. Potrebbe quindi trattarsi della stessa identica funzione o una simile con qualche modifica. <br></br>

Nel link ad un esempio di applicazione della libreria viene utilizzato il modulo `spelling` che però risulta assente. Dai commenti al post si suggerisce di utilizzare `suggest` che è anche la libreria usata nella documentazione di `pattern.en`.

In [122]:
%%time
for i in range(10):
    for sentence in imdb_df["Sentences4"].iloc[i]:
        temp = sentence.split()
        for word in temp:
            correct_word = suggest(word.lower()) 

Wall time: 11.9 s


<font size=3>

La terza libreria `SpellChecker` fa anch'essa riferimento al metodo visto precedentemente, ma a differenza degli altri prende in input una frase intera e non una singola parola. Non dovrebbero esserci differenze fondamentali nel modo in cui l'algoritmo lavora, ma si guadagna molto in termini di tempo. <br></br>

Un'altra differenza dalle due librerie prencedenti è l'obbligo di inizializzare la classe per poter applicare l'algorimo.

In [84]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [87]:
%%time
for i in range(10):
    for sentence in imdb_df["Sentences4"].iloc[i]:
        for word in sentence.split(" "):
            spell.correction(word)

Wall time: 5.78 s


<font size=3>

L'ultimo pacchetto è `SymSpell` introdotto recentemente, prende in prestito l'approccio precedente lo modifica rendendolo fino a [1000 volte](https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f) più veloce e più scalabile all'aumentare della distanza da considerare. <br></br>

L'<b>edit distance</b> rappresenta il numero di modifiche da effettuare per ricondursi ad un termine esistente. Tutti i metodi precedenti effettuavano le seguenti operazioni:
* Rimozione
* Trasposizione (di due caratteri adiacenti)
* Sostituzione
* Inserimento

Il valore dell'edit distance è calcolato in base alla formula di [Damerau–Levenshtein](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance), ogni operazione comporta l'aumento unitario dell'edit distance; se una stessa operazione viene ripetuta due volte, l'edit distance aumenta di altrettanti valori. Il valore dell'edit distance è fissato a $2$ nel caso delle prime due librerie, su `SpellChecker` è sempre due ma può essere ridotto a $1$. `SymSpell` permette invece di considerare qualsiasi valore come parametro in ingresso. <br></br>

1. Gli approcci precedenti creano un'elevata quantità di possibili termini da confrontare che esplode all'aumentare dell'edit distance massima.
2. Questo metodo utilizza solo la rimozione dei caratteri diminuendo notevolmente i tempi di lavoro. La rimozione viene effettuata in una fase di <b>pre-compilazione</b> sul dizionario e successivamente sui termini da correggere, in questa maniera si riduce drasticamente il numero di possibili candidati all'interno dell'edit distance massima.

Nella [repository GitHub](https://github.com/wolfgarbe/SymSpell) è presente in dettaglio la spiegazione del metodo e i confronti computazionali con altre tecniche. <br></br>

Per poter usufruire del metodo è necessario scaricare la libreria `pkg_resources`. Nelle altre righe viene inizializzato il dizionario, con le specifiche di pre-compilazione, che verrà successivamente passato alla funzione di correzione.

In [89]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [108]:
%%time
for i in range(10):
    for sentence in imdb_df["Sentences4"].iloc[i]:
        suggestions = sym_spell.lookup_compound(sentence, max_edit_distance=2, transfer_casing=True)

Wall time: 1.27 s


<font size=3>
    
I test effettuati confermano quello che gli articoli sostenevano, ovvero che l'ultimo metodo è estremamente più veloce di tutti gli altri. I primi due metodi sono pressoché identici, quindi è logico supporre che la funzione di base sia la stessa, con qualche modifica minore. Il terzo metodo nonostante sia basato sullo stesso funzionamento impiega la metà del tempo. Infine l'ultimo metodo è $5$ volte più veloce del terzo e $10$ rispetto ai primi due.

<font size=3>
    
L'ultimo algoritmo sembra l'unico a poter compiere il suo lavoro in tempi ragionevoli, sarà quindi l'unico metodo applicato in questa fase. Il codice per effettuare la spelling correction è il seguente:
* Viene inzializzata una lista in cui ogni elemento sarà una lista con dentro ogni frase di una recensione. Come già effettuato questa lista diventerà una nuova colonna del dataset.
* Si itera per ogni riga e si crea una lista temporanea. Queste liste conterranno le frasi corrette di una recensione e saranno ripulite ad ogni iterazione.
* Come fatto in precedenza si cerca se almeno una parola da correggere è all'interno della frase mediante la funzione `any`. In questo caso si utilizza `entity_diz_small` che contiene solamente le parole non riconosciute dall'algoritmo di NER.
    * Se si trova almeno una parola, tutta la frase viene passata al correttore e passando per ogni elemento dell'output si ha la stringa corretta che viene aggiunta alla lista temporanea.
    * Se nessuna parola è presente allora si aggiunge la frase originale.
* Alla fine di una iterazione la lista temporanea è aggiunta alla lista principale e si procede per via iterativa.

Per la correzione si utilizza una edit distance massima di $2$ e con il parametro `transfer_casing=True` si forza l'algoritmo ha mettere in output una parola che mantenga le stesse maiuscole/minuscole del termine originale (di default i risultati sono tutti i minuscolo).

In [118]:
symspell_list = []
for index, row in tqdm(imdb_df.iterrows()):
    temp_symspell = []
    for sentence in row["Sentences4"]:
        if any(word in entity_diz_small.keys() for word in sentence.split()):
            suggestions = sym_spell.lookup_compound(sentence, max_edit_distance=2, transfer_casing=True)
            for suggestion in suggestions:
                temp_symspell.append(suggestion.term)
        else:
            temp_symspell.append(sentence)
    symspell_list.append(temp_symspell)

784259it [4:03:54, 53.59it/s] 


<font size=3>
    
Per i futuri lavori sul notebook la lista viene salvata e caricata.

In [4]:
"""
with open('spelling', 'wb') as fp:
    pickle.dump(symspell_list, fp)
"""

with open('spelling', "rb") as input_file:
    symspell_list = pickle.load(input_file)

<font size=3>

Per avere una prima impressione di come abbia lavorato l'algoritmo si crea nuovamente un dizionario in cui si avranno come chiavi le parole non riconosciute dopo la correzione e come valore la loro frequenza.

In [136]:
%%time
unknown_words_spell = {}
for i in tqdm(range(len(symspell_list))):
    for sentence in symspell_list[i]:
        temp = sentence.split(" ")
        for word in temp:
            if word.lower() not in word_diz.keys():
                if word in unknown_words_spell.keys():
                    unknown_words_spell[word] += 1
                else:
                    unknown_words_spell[word] = 1

100%|███████████████████████████████████████████████████████████████████████| 784259/784259 [00:52<00:00, 14909.53it/s]


Wall time: 52.6 s


<font size=3>
    
Si stampa un confronto fra i $30$ termini più ricorrenti non riconosciuti in entrambi i dizionari. 

In [170]:
un = Counter(unknown_words).most_common(30)
un_spell = Counter(unknown_words_spell).most_common(30)
for i in range(len(un)):
    print('{:>25}  {:>12}'.format(str(un[i]), str(un_spell[i])))

           ('DVD', 28826)  ('filmmakers', 13303)
          ('IMDb', 17913)  ('kinda', 8453)
           ('CGI', 17391)  ('Netflix', 6747)
    ('filmmakers', 13175)  ('Bollywood', 6719)
        ('cliché', 12919)  ('indie', 6656)
        ('clichés', 9762)  ('blog', 5178)
          ('kinda', 8437)  ('WII', 4683)
      ('Bollywood', 6747)  ('UK', 3777)
        ('Netflix', 6682)  ('Pixar', 3244)
          ('indie', 6611)  ('Godzilla', 2751)
        ('clichéd', 6029)  ('DeNiro', 2709)
             ('ve', 5012)  ('SRK', 2494)
           ('http', 5012)  ('backstory', 2486)
             ('ww', 4849)  ('Cannes', 2485)
           ('WWII', 4706)  ('Facebook', 2453)
             ('UK', 4497)  ('wannabe', 2358)
          ('Pixar', 3591)  ('Fargo', 2277)
       ('blogspot', 3245)  ('website', 2225)
            ('SRK', 3083)  ('screenwriters', 2154)
         ('DeNiro', 2947)  ('Brokeback', 2028)
             ('WW', 2890)  ('bollywood', 2026)
       ('Godzilla', 2763)  ('BBC', 1869)
     ('Braveheart', 

<font size=3>

Le impressioni sulla correzione sono miste: da un lato si ha una frequenza inferiore dei termini, questo indica che diverse parole sono state modificate, anche se non si sa se correttamente, ma ci sono ancora dei termini che non dovrebbero esserci. Inoltre si è notato che la parola <b>DVD</b> è stata corretta con <b>DID</b> in quanto ritenuta la più vicina ($editdistance = 1$). <br></br>

Possono esserci diverse interpretazioni del risultato:
* Nonostante sia l'algoritmo che dovrebbe offrire il miglior risultato sia in termini di accuratezza sia in termini di prestazioni, non può essere preciso in tutti i casi. Anche per questo metodo l'accuratezza si aggira intorno al $70\%$, una discreta quantità di parole potrebbe essere sbagliata.
* La qualità della correzione si basa sul valore di edit distance massimo impostato. Come è spiegato dall'autore stesso del metodo in [un post](https://medium.com/p/8701fcd87a5f/responses/show) il valore $2$ è quello di default ed è l'unico realmente utilizzato insieme al $3$. I motivi di questa scelta sono:
    * La lunghezza media di una parola in inglese è $4.7$, un valore troppo alto restituisce troppi risultati, la maggior parte dei quali non inerenti.
    * Il $99\%$ dei termini è corretto con un edit distance di $2$.
    * L'aumento dell'edit distance produce più risultati e quindi aumenta i tempi di lavoro.
    
Per avere un'idea reale di quanto questo metodo abbia funzionato o meno si dovrà attendere le future analisi. Da qui in poi saranno considerati sia le recensioni pre-processate sia quelle con in più la spelling correction.

<br></br>
<br></br>
# [4. POS Tagging](#4)
<br></br>
## [4.1 NLP Tagger](#4.1)
<br></br>
<font size=3>
Per il POS tagging ci si affida ancora alla libreria `stanfordnlp`. La pipeline viene inizializzata con le due funzioni essenziali per il procedimento, la tokenizzazione ed appunto il tagging.

In [128]:
nlp = stanfordnlp.Pipeline(processors = "tokenize, pos")

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\Hp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\Hp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\Hp\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


<font size=3>
    
La struttura dell'algoritmo è la seguente:
* Viene passata una frase alla pipeline. Al suo interno sono effettuate le funzioni chiamate in precedenza.
* L'output rispecchia quello di ingresso, in questo caso sono state passate in input più frasi in una lista, l'output sono dei tag strutturari nella stessa maniera.
* Si accede alle singole parole delle singole frasi e al tag assegnato.

Sono restituiti due tipologie di tag come spiegato [nella documentazione](https://stanfordnlp.github.io/stanfordnlp/pos.html) di Stanford: 
* `UPOS`: Universal POS tags, è una tiplogia di tagging che prende in considerazione $17$ tipologie diverse; solitamente ogni tipologia corrisponde ad una parte del discorso con [qualche eccezione](https://www.sketchengine.eu/universal-pos-tags/).
* `XPOS`: Treebank-specific POS è una tipologia più specifica di tagging che identifica $36$ tag distinti. In questo caso sono identificati sottotipi di diverse parti del discordo. [A questo indirizzo](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) è presente la lista completa dei tag.

In [208]:
for sentence in imdb_df["Sentences4"].iloc[10]:
    doc = nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            print('{:<20s} {:<20s} {:<20s}'.format('word: ' + word.text, 'upos: ' + word.upos, 'xpos: ' + word.xpos))

word: Both           upos: CCONJ          xpos: CC            
word: Matt           upos: PROPN          xpos: NNP           
word: Damon          upos: PROPN          xpos: NNP           
word: and            upos: CCONJ          xpos: CC            
word: Bales          upos: PROPN          xpos: NNPS          
word: performances   upos: NOUN           xpos: NNS           
word: were           upos: AUX            xpos: VBD           
word: fantastic      upos: ADJ            xpos: JJ            
word: and            upos: CCONJ          xpos: CC            
word: brilliantly    upos: ADV            xpos: RB            
word: directed       upos: VERB           xpos: VBN           
word: by             upos: ADP            xpos: IN            
word: James          upos: PROPN          xpos: NNP           
word: by             upos: ADP            xpos: IN            
word: far            upos: ADV            xpos: RB            
word: his            upos: PRON           xpos: PRP$   

<font size=3>
    
Per l'aspect-based sentiment analysis non è necessario avere una visione dettagliata di quelli che possono essere i tag, negli articoli pubblicati solitamente si fa riferimento alle parti base del discorso. Viene comunque preferita la classificazione `XPOS` in quanto la notazione dei tag è realizzata in modo tale che i tag specifici di una determinata parte del discorso abbiano lo stesso inizio e delle lettere sono aggiunte in coda per i casi specifici. In questo modo è possibile avere tag specifici ma selezionando solo alla prima parte si ottiene la parte del discorso generale.

In [ ]:
tags = []
for index, row in tqdm(imdb_df.iterrows()):
    for sentence in row["Sentences4"]:
        temp_tags = []
        try:
            doc = nlp(sentence)
            for sent in doc.sentences:
                for word in sent.words:
                    temp_tags.append(word.xpos)
        except AssertionError:
            temp_tags.append("")
        tags.append(temp_tags)

<br></br>
## [4.2 NLTK + Server](#4.2)

<font size=3>

Il principale problema con il metodo precedente è la lentezza computazionale, circa una recensione e mezzo al secondo. Anche per questa procedura Stanford permette di utilizzare un proprio server per velocizzare le operazioni. <br></br>
Al contrario del metodo precedente per effettuare questa connessione è necessario utilizzare lo stesso algoritmo di POS tagging di Stanford che è però implementato nella libreria `nltk`:
* Basandosi su Java è necessario specificare il percorso in cui l'eseguibile è posizionato.
* Si seleziona il modello da importare e il file Java che permetterà di taggare le parole.
* Con la funzione `POSClient` presa sempre dal pacchetto `sner` si effettua la connessione al server.

Analogamente a quanto fatto per i NER si è dovuto creare una connessione mediante il prompt dei comandi. Il codice utlizzato è stato il seguente: `java -mx300m -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTaggerServer -model english-left3words-distsim.tagger -port 2020`. Gli elementi necessari sono rispettivamente: il file Java, uguale a quello specificato nella variable globale sul notebook, il server a cui collegarsi e il modello da utilizzare, anche in questo caso lo stesso specificato nel notebook.<br></br>

Nello .zip scaricato erano presenti due modelli e si utilizza quello meno prestazionale ma circa $10$ volte più veloce come [riportato sul sito](https://stanfordnlp.github.io/CoreNLP/memory-time.html#pos). Il modello scelto ottiene delle performance [leggermente inferiori](https://nlp.stanford.edu/software/pos-tagger-faq.html#h) rispetto a quello migliore ma è quello che garantisce il compromesso migliore tra accuratezza e velocità.

In [5]:
java_path = "C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath\\java.exe"
os.environ['JAVAHOME'] = java_path

model = 'english-left3words-distsim.tagger'
jar = 'stanford-postagger.jar'

pos_tagger = StanfordPOSTagger(model, jar, encoding='utf-8') 
tagger = POSClient(host='localhost', port=2020)

<font size=3>
    
La struttura dell'algoritmo è simile a quella usata in passato:
* Con `iterrows()` si passa una riga alla volta del dataset e si prende una frase alla volta delle recensioni.
* Se la frase non è vuota si applica il tagging e dall'output della funzione, una tupla, si accede al secondo elemento che è il tag.
* Questo tag viene aggiunto ad una lista temporanea e alla fine della frase si aggiunge tutto alla lista definita fuori dal loop. Nel caso ci sia una frase vuota si aggiunge un elemento nullo.

Per questioni di spazio si salva solamente il tag relativo ad ogni termine, per questa ragione è necessario prendere in considerazione anche le frasi rappresentate da stringhe vuote, perché in questa maniera si mantiene la corrispondenza di termine, in una colonna, e tag nella lista appena creata. <br></br>

Il risultato sono tante liste quante sono le frasi presenti nel dataset. Come già fatto in precedenza si utilizzerà lo slicing sulle liste per far combaciare più liste alla recensione relativa.

In [277]:
tags = []
for index, row in tqdm(imdb_df.iterrows()):
    for sentence in row["Sentences4"]:
        temp_tags = []
        if len(sentence) > 1:
            pos_tags = tagger.tag(sentence)
            for tag in pos_tags:
                temp_tags.append(tag[1])
        else:
            temp_tags.append("")
        tags.append(temp_tags)

784259it [3:13:08, 67.68it/s] 


<font size=3>
    
Viene replicata la stessa operazione sulle parole corrette. Dato che la lista ottenuta non fa parte del dataset si utilizza un generico loop. L'unica differenza con il codice precedente è che in questo caso si è voluto mantenere la struttura originale:
* Ad ogni recensione si crea una nuova lista, elemento inedito rispetto al codice precedente.
* Per ogni frase si crea ancora una lista temporanea in cui sono salvati i tag. A differenza di prima, alla fine della frase l'insieme dei tag è convertito in una stringa con la funzione `split()` e gli elementi sono delimitati da uno spazio come se fossero parole. <br></br> Con questa logica con un futuro `split()` sarà semplice avere la corrispondenza esatta fra parole in una lista e tag nell'altra.

In [18]:
tags_spell = []
for i in tqdm(range(len(symspell_list))):
    review_tags = []
    for sentence in symspell_list[i]:
        temp_tags = []
        if len(sentence) > 1:
            pos_tags = tagger.tag(sentence)
            for tag in pos_tags:
                temp_tags.append(tag[1])
        else:
            temp_tags.append("")
        idx = " ".join(temp_tags)
        review_tags.append(idx)
    tags_spell.append(review_tags)

100%|████████████████████████████████████████████████████████████████████████| 784259/784259 [3:08:01<00:00, 69.52it/s]


<font size=3>
    
Anche in questo caso per gli ingenti tempi computazali gli output sono salvati con pickle ed è possibile ricaricarli nel loro formato originale.

In [278]:
"""
with open('tags_sent', 'wb') as fp:
    pickle.dump(tags, fp)
"""
    
with open('tags_sent', "rb") as input_file:
    tags_sent = pickle.load(input_file)

In [19]:
"""
with open('tags_spell', 'wb') as fp:
    pickle.dump(tags_spell, fp)
"""
    
with open('tags_spell', "rb") as input_file:
    tags_spell = pickle.load(input_file)